In [1]:
# Importing library
import pandas as pd
import os
import glob
import warnings
warnings.filterwarnings('ignore')
import math 
import mysql.connector as msql
import re

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Displaying all columns and rows
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [4]:
# Taking files from define path which are in csv format
path = r'C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\04 Incubator - Curriculum Cohort\Incubator 6.0 UK\data_files\04_Source_Kalpana Assignment Review'

# Change the current working directory to the specified path
os.chdir(path)

# Use glob to get a list of file paths for all CSV files in the current directory
file_paths = glob.glob('*.csv')

In [5]:
# Create an empty list to store the keywords
keywords = []

# Create an empty dictionary to store the dataframes
dataframes = {}

# Iterate through each file path in the list of file paths
for file in file_paths:
    # Find the starting position of the keyword by searching for 'Assignment_'
    start = file.find('Assignment_') + len('Assignment_')
    
    # Find the ending position of the keyword by searching for '.csv'
    end = file.rfind('.csv')
    
    # Extract the keyword from the file path, removing leading/trailing spaces, and replacing spaces with underscores
    keyword = file[start:end].strip().replace(' ', '_')
    
    # Add the extracted keyword to the list of keywords
    keywords.append(keyword)
    
    # Read the CSV file and store it in a dataframe with the keyword as the key in the dataframes dictionary
    dataframes[keyword] = pd.read_csv(file)


In [6]:
# List of keyword of our csv file present in our folder
keywords

['Career_Action_Plan',
 'Career_Exploration',
 'CV__Resume',
 'Goal_Setting',
 'SMART_goals']

In [7]:
# Iterate through each keyword in the list of keywords
for keyword in keywords:
    # Assign the corresponding dataframe to a variable with the same name as the keyword
    # This is done by using the globals() function to create a new variable in the global namespace
    globals()[keyword] = dataframes[keyword]

    # We have use golabl function to create dataframe of keyword so it should take name of dataframe as keyword. Like Swot will be name of datafarme.

In [8]:
# Reading Kalpana SHE for STEM Incubator file from Source files

directory_path =(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\04 Incubator - Curriculum Cohort\Incubator 6.0 UK\data_files\02_Source_Kalpana Incubator")

csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(directory_path, file)
    Kalpana = pd.read_csv(file_path,usecols=['Email'])
    print(f"Data from {file}:")

Data from She for STEM - Uttarakhand.csv:


In [9]:
# Create an empty dataframe to store the final results
final_df = pd.DataFrame()

# Create a dictionary to store comments for each keyword
comments_dict = {}

# Iterate through each keyword in the list of keywords
for keyword in keywords:
    # Filter the columns based on specific conditions...
    filtered_columns = [col for col in globals()[keyword] if ('message' in col) or ('email' in col) or (col == 'status')]

    new_df = globals()[keyword][filtered_columns]
    
    message_columns = [col for col in new_df.columns if 'message' in col]
    new_df[message_columns] = new_df[message_columns].fillna('')
    
    # Create a new 'Comment' column by joining the values of the 'message' columns for each row
    comment_col_name = 'Comment_' + keyword
    new_df[comment_col_name] = new_df[message_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    
    # Store the comments column name in the dictionary
    comments_dict[keyword] = comment_col_name
    
    # Modify the 'Email' column by removing a specific prefix from the values
    new_df['Email'] = new_df['user/email'].str.replace('vigyanshaalainternational1617-', '')
    
    # Rename the 'status' column to the keyword
    new_df = new_df.rename(columns={'status': keyword})
    
    # Select the desired columns for the new dataframe
    selected_columns = ['Email', keyword, comment_col_name]
    new_df = new_df[selected_columns]
    
    # If the final dataframe is empty, assign it as the new_df
    if final_df.empty:
        final_df = new_df.copy()
    # If the final dataframe is not empty, merge the new_df with the existing final dataframe using an outer join
    else:
        final_df = pd.merge(final_df, new_df, on='Email', how='outer', suffixes=('', f'_{keyword}'))

# Fill NaN values in the final dataframe
final_df = final_df.fillna('')

# Now, final_df contains all unique emails from all keywords


In [10]:
# Copying dataframe for part c of code
final_df1 = final_df.copy()

In [11]:
# Create a list of columns to assign, excluding 'Email' and columns starting with 'Comment'
assign = [col for col in final_df.columns if col != 'Email' and not col.startswith('Comment')]


In [12]:
# Remove leading/trailing spaces in the selected columns
final_df[assign] = final_df[assign].applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [13]:
# To get unique values of dataframe to see what columns it consist to give 'marks'
final_df['Goal_Setting'].unique()

array(['reviewed', 'under review', '', 'rejected'], dtype=object)

# Below you can give marks for Assignment 👇⬇️

In [14]:
replace_dict = {'under review': 30, 'reviewed': 100,'rejected': 80,'': 0}


In [15]:
# Replace values in selected columns
final_df[assign] = final_df[assign].replace(replace_dict)

In [16]:
# Convert the selected columns to float
final_df[assign] = final_df[assign].astype(float)

In [17]:
# Filling NaN Values with zero
final_df[assign]=final_df[assign].fillna(0)

In [18]:
# Calculate the sum of selected columns
sum_of_scores = final_df[assign].sum(axis=1)

# Count the number of selected columns
num_of_columns = len(assign)

# Calculate the average score
final_df['Assignment_Score'] = sum_of_scores / num_of_columns


In [19]:
# Rounding Off values of Assignment score to two digits
final_df['Assignment_Score']=final_df['Assignment_Score'].round(2)

In [20]:
# Filling NaN Values with empty
final_df=final_df.fillna('')

In [21]:
final_df = final_df.replace('Successfully Submitted ', '# ', regex=True)


In [22]:
# Iterate through all columns in the dataframe
for col in final_df.columns:
    # Check if column name contains '.'
    if '.' in col:
        # Replace '.' with '_' and remove ' ' only where '.' is present in the column name
        new_col = col.replace('.', '_').replace(' ', '') if '.' in col else col
        # Rename the column
        df.rename(columns={col: new_col}, inplace=True)


In [23]:
final_df = pd.merge(final_df, Kalpana, on='Email', how='inner')


In [24]:
final_df.shape

(81, 12)

### The step remaing here is renaming i.e while putting to our mysql

In [25]:
# Get the list of column names in the final_df
column_names = final_df.columns

# Iterate through the column names and rename columns starting with "Comment"
for col in column_names:
    if col.startswith("Comment"):
        new_col_name = "Comment"
        final_df = final_df.rename(columns={col: new_col_name})



In [26]:
# Checking the renaming column name
final_df.columns

Index(['Email', 'Career_Action_Plan', 'Comment', 'Career_Exploration',
       'Comment', 'CV__Resume', 'Comment', 'Goal_Setting', 'Comment',
       'SMART_goals', 'Comment', 'Assignment_Score'],
      dtype='object')

In [27]:
# Renaming the column to put in datafarme 
new_names = {
    'SWOT': 'SWOT',
    'Career_Action_Plan_(CAP)': 'Career_Action_Plan',
    'Assignment_Goal_Setting' : 'Goal_Setting',
    'Critical_thinking': 'Critical_Thinking',
    'Masteclass_1_-_Career_Exploration': 'Career_Exploration',
    'Masterclass_-_Academic_CV_and_Industry_Resume': 'CV_Resume',
    'Networking__Build_a_World-Class_LinkedIn_Profile': 'LinkedIn_Profile',
    'Planning_&_Applying_for_Masters_in_India_&_Abroad': 'Planning_Masters',
    'RIASEC_personality_test': 'RIASEC',
    'SMART_goal': 'SMART_Goal',
    'Career_Action_Plan':'Career_Action_Plan',
    'CV__Resume' : 'CV_Resume',
    'SMART_goals': 'SMART_Goal'
}

for col in final_df.columns:
    if col in new_names:
        final_df.rename(columns={col: new_names[col]}, inplace=True)


In [28]:
# Checking the renaming column name
final_df.columns

Index(['Email', 'Career_Action_Plan', 'Comment', 'Career_Exploration',
       'Comment', 'CV_Resume', 'Comment', 'Goal_Setting', 'Comment',
       'SMART_Goal', 'Comment', 'Assignment_Score'],
      dtype='object')

In [29]:
# Renaming the 'comment' in our given format
cols = final_df.columns.tolist()
new_cols = []
prev_col = ''

for col in cols:
    if col == 'Comment':
        new_cols.append(f'Comments_{prev_col}')
    else:
        new_cols.append(col)
    prev_col = col

final_df.columns = new_cols


In [30]:
# Checking the renaming column name
final_df.columns

Index(['Email', 'Career_Action_Plan', 'Comments_Career_Action_Plan',
       'Career_Exploration', 'Comments_Career_Exploration', 'CV_Resume',
       'Comments_CV_Resume', 'Goal_Setting', 'Comments_Goal_Setting',
       'SMART_Goal', 'Comments_SMART_Goal', 'Assignment_Score'],
      dtype='object')

In [31]:
# Function to clean email addresses
def clean_email(email):

    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
final_df['Email'] = final_df['Email'].apply(clean_email)

### Adding Genral Info 

In [32]:
# Import Genral Info
gi = pd.read_csv(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\04 Incubator - Curriculum Cohort\Incubator 6.0 UK\output\General_Info.csv")

In [33]:
gi = gi[['Email', 'Name', 'Phone', 'Name_of_College_University','Currently_Pursuing_Degree']].copy()

In [34]:
# Merge with gi
output = pd.merge(gi, final_df, on='Email', how='outer')

In [35]:
# Assuming df is your final DataFrame
output_directory = r'C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\08 Support Code\02 Edu Team Notation Code\Output'

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

output_path = os.path.join(output_directory, 'Assignment_Review.csv')
output.to_csv(output_path, index=False)

In [36]:
#output.to_csv(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\04 Incubator - Curriculum Cohort\Incubator 6.0 UK\output\Assignment_Review.csv",mode='w',index=False)

In [37]:
# Renaming the column to put in datafarme 
new_names = {
    'Assignment_CAP': 'Career_Action_Plan',
    'Comments_Assignment_CAP': 'Comments_Career_Action_Plan',
    'Assignment_Career_Exploration' : 'Career_Exploration',
    'Comments_Assignment_Career_Exploration': 'Comments_Career_Exploration',
    'Assignment_Goal_Setting': 'Goal_Setting',
    'Comments_Goal_Setting': 'Comments_Goal_Setting',    
    'Goal_Setting': 'Goal_Setting',
    'Comments_Goal_Setting': 'Comments_Goal_Setting',
    'Assignment_LinkedIn': 'LinkedIn_Profile',
    'Comments_Assignment_LinkedIn' : 'Comments_LinkedIn_Profile',
    'Planning_&_Applying_for_Masters_in_India_&_Abroad': 'Planning_Masters',
    'RIASEC_personality_test': 'RIASEC',
    'SMART_goal': 'SMART_Goal',
    'Assignment_Searching_&_Securing_Internship' : 'Internship_Searching',
    'Comments_Assignment_Searching_&_Securing_Internship' : 'Comments_Internship_Searching',
    'Assignment_Resume' : 'CV_Resume',
    'Comments_Assignment_Resume' : 'Comments_CV_Resume',
    'Assignment_SMART_goals' : 'SMART_Goal',
    'Comments_Assignment_SMART_goals' : 'Comments_SMART_Goal',
    'Assignment_CV__Resume' :'CV_Resume',
    'Comments_Assignment_CV__Resume' : 'Comments_CV_Resume',
    'Assignment_CV_Resume' : 'CV_Resume',
    'Comments_Assignment_CV_Resume' : 'Comments_CV_Resume'
}

for col in final_df.columns:
    if col in new_names:
        final_df.rename(columns={col: new_names[col]}, inplace=True)

In [38]:
final_df.columns

Index(['Email', 'Career_Action_Plan', 'Comments_Career_Action_Plan',
       'Career_Exploration', 'Comments_Career_Exploration', 'CV_Resume',
       'Comments_CV_Resume', 'Goal_Setting', 'Comments_Goal_Setting',
       'SMART_Goal', 'Comments_SMART_Goal', 'Assignment_Score'],
      dtype='object')

### Storing data on MySQL

In [39]:
# Connecting to MySQL Database
conn= msql.connect(host='localhost',user='root',password="VS@123",database="she_for_stem",auth_plugin='mysql_native_password')
cursor =conn.cursor()

In [40]:
# Your existing code for inserting data into the database table
for i, row in final_df.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    columns = ','.join(final_df.columns)
    placeholders = ','.join(['%s']*len(row))
    # Construct the INSERT query with ON DUPLICATE KEY UPDATE clause
    query = f"INSERT INTO she_for_stem.11_incubator_assignment_monitoring ({columns}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE "
    query += ", ".join([f"{col}=VALUES({col})" for col in final_df.columns if col != 'Email'])
    # Execute the query
    cursor.execute(query, tuple(row))


In [41]:
conn.commit()

In [42]:
output.head()

,Email,Name,Phone,Name_of_College_University,Currently_Pursuing_Degree,Career_Action_Plan,Comments_Career_Action_Plan,Career_Exploration,Comments_Career_Exploration,CV_Resume,Comments_CV_Resume,Goal_Setting,Comments_Goal_Setting,SMART_Goal,Comments_SMART_Goal,Assignment_Score
0,nehakr2022@gmail.com,Neha Kumari,8.126394e+09,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 3rd year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ankitapandey.bgr2005@gmail.com,Ankita Pandey,8.394071e+09,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 3rd year,30.0,Successfully Submitted,100.0,# You passed this. Try to explore more details...,0.0,,100.0,"# You need to bring clarity in your 3yrs, 5yrs...",100.0,# This is good. you have understood the concep...,66.0
2,ridhimadixit1409@gmail.com,Ridhima Dixit,9.359899e+09,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 3rd year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,guptaminkashi153@gmail.com,Minakshi gupta,9.760190e+09,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 3rd year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ankitapargai@gmail.com,Ankita Pargai,9.548626e+09,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 4th year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Part B

In [43]:
# Create a dictionary to store comments for each keyword
comments_dict = {}

# List to hold the extracted data for each keyword
output = []

# Maximum possible submissions, adjust this based on actual data
max_submissions = 10

# Iterate through each keyword in the list of keywords
for keyword in keywords:
    # We will use try-except blocks to handle missing columns dynamically
    for index, row in globals()[keyword].iterrows():
        sub_number = 1
        for i in range(max_submissions):  # Loop through potential submission numbers
            try:
                file_col = f'data/{i}/fileName'
                status_col = f'data/{i+1}/status'
                message_col = f'data/{i+1}/message'
                # Check if the file column exists and is not empty
                if file_col in globals()[keyword].columns and pd.notna(row[file_col]):
                    output.append({
                        'Assignment_File': keyword,  # Add the keyword to the Assignment column
                        'File_Name': row[file_col],
                        'File_Status': row[status_col] if status_col in globals()[keyword].columns else 'N/A',
                        'Submission_Number': sub_number,
                        'Email_ID': row['user/email'],
                        'Comment': row[message_col] if message_col in globals()[keyword].columns else 'N/A'
                    })
                    sub_number += 1
            except KeyError:
                # Ignore missing columns or continue if the column doesn't exist
                continue

# Create a dataframe with the extracted information for this keyword
keyword_df = pd.DataFrame(output, columns=['Assignment_File','File_Name', 'File_Status', 'Submission_Number', 'Email_ID', 'Comment'])

# Modify the 'Email' column by removing a specific prefix from the values
keyword_df['Email_ID'] = keyword_df['Email_ID'].str.replace('vigyanshaalainternational1617-', '')

# Fill NaN values in the final dataframe with default values
keyword_df['Comment'].fillna('Successfully Submitted', inplace=True)
keyword_df['File_Status'].fillna('under review', inplace=True)




In [44]:
# Function to check for special characters in the last 5 characters of a string
def extract_special_chars(value):
    special_chars = "!@#$%^&*?+"
    found_chars = ''.join([char for char in value[-5:] if char in special_chars])
    return found_chars if found_chars else None

# Apply the function to the 'comment' column and create a new 'notation' column
keyword_df['Notation'] = keyword_df['Comment'].apply(extract_special_chars)

In [45]:
# Function to determine 'user' and 'marks' based on 'Notation' column
def extract_user_marks(Notation):
    # Skip the row if Notation is None or empty
    if pd.isna(Notation) or Notation == '':
        return pd.Series([None, None])
    
    user = 'Other'
    marks = None
    
    # Determine the user based on the first character of Notation
    if len(Notation) > 0:  # Check if there's at least one character
        if Notation[0] == '#':
            user = 'Dipika'
        elif Notation[0] == '!':
            user = 'Salini'

    # Dictionary for marks
    marks_dict = {'!': 1, '@': 2, '#': 3, '$': 4, '%': 5, '^': 6, '&': 7, '*': 8, '?': 9, '+': 0, '!+': 10}
    
    # Check for the special '!+' case first
    if len(Notation) > 2 and Notation[:2] == '!+':
        marks = marks_dict['!+']
    else:
        # Get marks based on second and third characters if present
        marks = marks_dict.get(Notation[1], 0) if len(Notation) > 1 else 0
        marks += marks_dict.get(Notation[2], 0) if len(Notation) > 2 else 0
    
    return pd.Series([user, marks])

# Apply the function to create 'user' and 'marks' columns, skipping None values
keyword_df[['User_Name', 'Marks']] = keyword_df['Notation'].apply(extract_user_marks)


In [46]:
keyword_df.head()

,Assignment_File,File_Name,File_Status,Submission_Number,Email_ID,Comment,Notation,User_Name,Marks
0,Career_Action_Plan,CAP Assignment_Template -1.pptx,N/A,1,ankitapandey.bgr2005@gmail.com,N/A,None,None,NaN
1,Career_Action_Plan,CAP ASSIGNMENT_DONE.pptx,N/A,1,priyanagarkoti1712@gmail.com,N/A,None,None,NaN
2,Career_Action_Plan,assignment CAP.pdf,N/A,1,rituhada28251@gmail.com,N/A,None,None,NaN
3,Career_Action_Plan,CAP Assignment_Template .pptx,N/A,1,ruchirawat1825@gmail.com,N/A,None,None,NaN
4,Career_Action_Plan,preeti.pptx,N/A,1,nariyalpriti27@gmail.com,N/A,None,None,NaN


# 👇 Change Cohort Name Below 👇

In [47]:
# Add new column 'xyz' with the value 'Incubator_6'
keyword_df['Cohort'] = 'Incubator_6'

In [48]:
# Remove 'Assignment_' prefix if it exists in the values of 'Assignment_File' column
keyword_df['Assignment_File'] = keyword_df['Assignment_File'].str.replace('Assignment_', '')


In [49]:
# Drop the column 'Notation'
keyword_df = keyword_df.drop(columns=['Notation'])

In [50]:
# Count occurrences of each user name
user_counts = keyword_df['User_Name'].value_counts()

# Print the counts in the desired format
for user, count in user_counts.items():
    cohort = keyword_df.loc[keyword_df['User_Name'] == user, 'Cohort'].iloc[0]  # Assuming there's a 'Cohort' column
    print(f"{user} has checked {count} Assignments in {cohort}")


Dipika has checked 188 Assignments in Incubator_6
Salini has checked 44 Assignments in Incubator_6
Other has checked 15 Assignments in Incubator_6


In [51]:
keyword_df.to_csv(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\08 Support Code\02 Edu Team Notation Code\Stem_check.csv",mode='w',index=False)

In [52]:
keyword_df.duplicated().sum()

0

In [53]:
# Assuming df is your final DataFrame
output_directory = r'C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\08 Support Code\02 Edu Team Notation Code\Output'

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

output_path = os.path.join(output_directory, 'Stem_check.csv')
keyword_df.to_csv(output_path, index=False)

In [54]:
#keyword_df.to_csv(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\08 Support Code\02 Edu Team Notation Code\Stem_check.csv",mode='w',index=False)

# Part C

In [55]:
final_df1.head()

,Email,Career_Action_Plan,Comment_Career_Action_Plan,Career_Exploration,Comment_Career_Exploration,CV__Resume,Comment_CV__Resume,Goal_Setting,Comment_Goal_Setting,SMART_goals,Comment_SMART_goals
0,ankitapandey.bgr2005@gmail.com,under review,Successfully Submitted,reviewed,Successfully Submitted You passed this. Try to...,,,reviewed,Successfully Submitted You need to bring clari...,reviewed,Successfully Submitted This is good. you have ...
1,priyanagarkoti1712@gmail.com,under review,Successfully Submitted,under review,Successfully Submitted,,,reviewed,Successfully Submitted You need to bring clari...,reviewed,Successfully Submitted This is good. you have ...
2,rituhada28251@gmail.com,under review,Successfully Submitted,reviewed,Successfully Submitted You passed this. Try to...,reviewed,Successfully Submitted The RESUME must be subm...,reviewed,Successfully Submitted You need to bring clari...,reviewed,Successfully Submitted This is good. you have ...
3,ruchirawat1825@gmail.com,under review,Successfully Submitted,,,reviewed,Successfully Submitted The RESUME must be subm...,reviewed,Successfully Submitted Awesome! You understood...,reviewed,Successfully Submitted This is ok. you can mak...
4,nariyalpriti27@gmail.com,under review,Successfully Submitted,,,reviewed,"Successfully Submitted Good! Just one thing, y...",reviewed,Successfully Submitted Good job. You can add m...,reviewed,"Successfully Submitted It is long term goal, b..."


In [56]:
# Function to clean email addresses
def clean_email(email):
    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
final_df1['Email'] = final_df1['Email'].apply(clean_email)

In [57]:
# Merge with gi
final_df1 = pd.merge(gi, final_df1, on='Email', how='outer')

In [58]:
# Create a list of columns to assign, excluding 'Email' and columns starting with 'Comment'
assign1 = [col for col in final_df1.columns if col != 'Email' and not col.startswith('Comment')]


In [59]:
# Remove leading/trailing spaces in the selected columns
final_df1[assign1] = final_df1[assign1].applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Below you can give names for Assignment 👇⬇️

In [60]:
replace_dict = {'under review': 'Under review', 'reviewed': 'Accepted','rejected': 'Rejected','': "Not Submitted"}

In [61]:
# Replace values in selected columns
final_df1[assign1] = final_df1[assign1].replace(replace_dict)

In [62]:
# Filling NaN Values with empty
final_df1=final_df1.fillna('Not Submitted')

In [63]:
# Changing the comment
final_df1 = final_df1.replace('Successfully Submitted ', '# ', regex=True)


In [64]:
# Iterate through all columns in the dataframe
for col in final_df1.columns:
    # Check if column name contains '.'
    if '.' in col:
        # Replace '.' with '_' and remove ' ' only where '.' is present in the column name
        new_col = col.replace('.', '_').replace(' ', '') if '.' in col else col
        # Rename the column
        df.rename(columns={col: new_col}, inplace=True)


In [65]:
# Cheaking the columns present in the dataframe
final_df1.columns

Index(['Email', 'Name', 'Phone', 'Name_of_College_University',
       'Currently_Pursuing_Degree', 'Career_Action_Plan',
       'Comment_Career_Action_Plan', 'Career_Exploration',
       'Comment_Career_Exploration', 'CV__Resume', 'Comment_CV__Resume',
       'Goal_Setting', 'Comment_Goal_Setting', 'SMART_goals',
       'Comment_SMART_goals'],
      dtype='object')

In [66]:
# Assuming df is your final DataFrame
output_directory = r'C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\08 Support Code\02 Edu Team Notation Code\Output'

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

output_path = os.path.join(output_directory, 'Student_Assignment.csv')
final_df1.to_csv(output_path, index=False)

# Part D

In [67]:
final_df1

,Email,Name,Phone,Name_of_College_University,Currently_Pursuing_Degree,Career_Action_Plan,Comment_Career_Action_Plan,Career_Exploration,Comment_Career_Exploration,CV__Resume,Comment_CV__Resume,Goal_Setting,Comment_Goal_Setting,SMART_goals,Comment_SMART_goals
0,nehakr2022@gmail.com,Neha Kumari,8126393572.0,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 3rd year,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted
1,ankitapandey.bgr2005@gmail.com,Ankita Pandey,8394071318.0,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 3rd year,Under review,Successfully Submitted,Accepted,# You passed this. Try to explore more details...,Not Submitted,,Accepted,"# You need to bring clarity in your 3yrs, 5yrs...",Accepted,# This is good. you have understood the concep...
2,ridhimadixit1409@gmail.com,Ridhima Dixit,9359898815.0,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 3rd year,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted
3,guptaminkashi153@gmail.com,Minakshi gupta,9760189834.0,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 3rd year,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted
4,ankitapargai@gmail.com,Ankita Pargai,9548625626.0,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 4th year,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted
5,randeepkaurrandeep700@gmail.com,Ramandeep Kaur,8958494831.0,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 3rd year,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted
6,akshirajput578@gmail.com,Akshi Chauhan,9105023968.0,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 4th year,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted
7,ritusah962@gmail.com,Ritu kumari,9634155563.0,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 3rd year,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted
8,nancymahar2012@gmail.com,Nancy Mahar,8433113631.0,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 3rd year,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted
9,yashaswinishukla77@gmail.com,Yashaswini Shukla,7017699023.0,Dr. A.P.J. Abdul Kalam Institute of Technology...,B.E./B.Tech 3rd year,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted,Not Submitted


In [68]:
# Creating a DataFrame
clg_df = pd.DataFrame(final_df1)

# Example of main imported data with a column named 'college'
main_imported_data = {
    'college': final_df1['Name_of_College_University'],
}

# Including the 'college' column from the main imported data
clg_df['college'] = main_imported_data['college']

# Selecting columns that start with "Assignment_" and including the "college" column
selected_columns = clg_df.filter(regex='^(Assignment_|college)')

# Displaying the selected columns
selected_columns.head()

,college
0,Dr. A.P.J. Abdul Kalam Institute of Technology...
1,Dr. A.P.J. Abdul Kalam Institute of Technology...
2,Dr. A.P.J. Abdul Kalam Institute of Technology...
3,Dr. A.P.J. Abdul Kalam Institute of Technology...
4,Dr. A.P.J. Abdul Kalam Institute of Technology...


In [69]:
#selected_columns['Assignment_Goal_Setting'].unique()

In [70]:

# Create a function to count the number of 'Accepted' and 'Rejected' assignments in each column
def count_accepted_rejected(row):
    accepted_count = (row == 'Accepted').sum()
    rejected_count = (row == 'Rejected').sum()
    submission_count=accepted_count+rejected_count
    return submission_count
    
# Group the data by 'college' and apply the function to get the counts for each college
grouped_accepted_rejected_counts = selected_columns.groupby('college').apply(lambda x: x.drop('college', axis=1).apply(count_accepted_rejected))

# Display the counts of 'Accepted' and 'Rejected' assignments for each college and each assignment column
grouped_accepted_rejected_counts


""
college
"Dev Bhoomi Uttarakhand University, Dehradun"
"Dr. A.P.J. Abdul Kalam Institute of Technology, Tanakpur"
G B Pant Institute of Engineering & Technology
G.P.G.C.Berinag (Stemchampions)
"Institute of Technology, Gopeshwar"
"JBIT, Dehradun"
LSM PG CAMPUS PITHORAGARH (Stemchampions)
"Nanhi Pari Seemant Institute of Technology, Pithoragarh"
Not Submitted


In [71]:
college=selected_columns['college']

# Creating a DataFrame
dp = pd.DataFrame(final_df1)

# Using the melt function to merge the columns into a single column
merged_data = dp.melt(value_vars=dp.filter(regex='^(Assignment_|college)'), value_name='Assignment').drop(columns='variable')


# Displaying the merged data with a single 'Assignment' column
merged_data


,Assignment


In [72]:
final_df1 = selected_columns   # storing selected column dataframe under variable data
dp = pd.DataFrame(final_df1)   # Creating a DataFrame

college = dp['college']  # Extracting the 'college' column

# Using the melt function to merge the columns into a single column and including the 'college' column
merged_data = dp.melt(value_vars=selected_columns, 
                      var_name='Assignment', value_name='Score',
                      id_vars='college')

# Displaying the merged data with the 'Assignment' and 'college' columns
merged_data


,college,Assignment,Score


In [73]:
merged_data['Score'].value_counts()

Series([], Name: count, dtype: int64)

In [74]:
len(merged_data['college'].unique())

0

In [75]:
# Value counts of items present in the 'Score' column
score_value_counts = merged_data['Score'].value_counts()

# Displaying the value counts of items in the 'Score' column
print(score_value_counts)


Series([], Name: count, dtype: int64)


In [76]:
# Group the data by 'Score' and 'College' and then obtain the value counts
score_college_value_counts = merged_data.groupby(['Score', 'college']).size().reset_index(name='Count')

# Displaying the value counts of items in the 'Score' column grouped by 'College'
print(score_college_value_counts)


Empty DataFrame
Columns: [Score, college, Count]
Index: []


In [77]:
# Value counts of items present in the 'Score' column grouped by the name of the college
score_value_counts_by_college = merged_data.groupby('college')['Score'].value_counts()

# Displaying the value counts of items in the 'Score' column grouped by college name
print(score_value_counts_by_college)


Series([], Name: count, dtype: int64)


In [78]:
# Creating a pivot table to get counts of different 'Score' categories
pivot_table = merged_data.pivot_table(index='college', columns='Score', aggfunc='size', fill_value=0)

# Renaming the columns for better readability
pivot_table.rename(columns={'not submitted': 'Not Submitted Count', 
                            'submitted': 'Submitted Count', 
                            'accepted': 'Accepted Count', 
                            'rejected': 'Rejected Count'}, inplace=True)

# Displaying the pivot table with the specified column names
pivot_table.head()


Score
college


In [79]:
# Calculating the acceptance rate
pivot_table['Acceptance Rate(in Percentage)'] = ((pivot_table['Accepted'] / (pivot_table['Accepted'] + pivot_table['Rejected'])).round(4))*100
pivot_table['Total Assignment']=(pivot_table['Accepted'] + pivot_table['Rejected'] + pivot_table['Under review'])

# Reorder the columns with the "Not Submitted" column moved after the "Total Assignment" column
pivot_table = pivot_table[['Accepted', 'Rejected', 'Under review','Total Assignment','Not Submitted' ,'Acceptance Rate(in Percentage)']]

# Displaying the updated and organized tabular data
pivot_table



KeyError: 'Accepted'

In [ ]:
merged_df2=pd.merge(grouped_accepted_rejected_counts,pivot_table,on='college',how='inner')

In [ ]:
merged_df2

In [ ]:
gi.columns

In [ ]:

college_student_count = gi.groupby(['Name_of_College_University', 'Name']).size().reset_index(name='count')

# Replace 'college' and 'student_name' with the actual column names in your DataFrame
# The college_student_count DataFrame will contain the count of students for each college and student name
college_student_count


In [ ]:

# Group by the college name and count the number of students in each college
college_student_count = gi.groupby('Name_of_College_University')['Name'].count().reset_index(name='Student_Count')

# Print the count of students in each unique college
college_student_count


In [ ]:
college_student_count= college_student_count.rename(columns={'Name_of_College_University': 'college'})


In [ ]:
merged_df3=pd.merge(college_student_count,merged_df2,on='college',how='inner')

In [ ]:
merged_df3

### Assignment wise acceptance rate:

In [ ]:
# Assuming merged_data is your DataFrame containing the data
# Splitting the 'Score' column into 'Accepted' and 'Rejected'
merged_data['Accepted'] = merged_data['Score'].apply(lambda x: 1 if x == 'Accepted' else 0)
merged_data['Rejected'] = merged_data['Score'].apply(lambda x: 1 if x == 'Rejected' else 0)
merged_data['Under_Review'] = merged_data['Score'].apply(lambda x: 1 if x == 'Under_Review' else 0)


# Grouping by 'college' and 'Assignment' and summing the 'Accepted' and 'Rejected' columns
result = merged_data.groupby(['college','Assignment']).agg({'Accepted': 'sum', 'Rejected': 'sum','Under_Review':'sum'}).reset_index()

result


In [ ]:
# Grouping the DataFrame by 'College' and 'Assignment' and aggregating accepted and rejected counts
grouped_data = result.groupby(['college', 'Assignment']).agg({'Accepted': 'sum', 'Rejected': 'sum','Under_Review':'sum'})

grouped_data

In [ ]:
# Pivot the DataFrame to display assignments in a horizontal format
result = result.pivot(index='college', columns='Assignment', values=['Accepted', 'Rejected','Under_Review']).fillna(0)

# Flatten the column index
result.columns = ['_'.join(col).strip() for col in result.columns.values]

# Reset the index to make 'College' a column again
result1 = result.reset_index()

result1.head()

In [ ]:
# List of assignment columns to calculate total assignment count and acceptance rates for
assignment_columns = ['SMART_goals', 'Goal_Setting', 'CV_Resume']

# Calculate total assignment count and acceptance rates for each assignment column
for assignment in assignment_columns:
    accepted_col = f'Accepted_Assignment_{assignment}'
    rejected_col = f'Rejected_Assignment_{assignment}'
    under_review_col = f'Under_Review_Assignment_{assignment}'
    total_assignment_count_col = f'Total_Assignment_count_Assignment_{assignment}'
    acceptance_rate_col = f'Acceptance_Rate_Assignment_{assignment}'
    
    result1[total_assignment_count_col] = result1[accepted_col] + result1[rejected_col] + result1[under_review_col]
    result1[acceptance_rate_col] = ((result1[accepted_col] / (result1[accepted_col] + result1[rejected_col])).round(4))*100

result1


In [ ]:
merged_df4=pd.merge(college_student_count,result1,on='college',how='inner')

In [ ]:
merged_df4.head()

In [ ]:
merged_df4.columns

In [ ]:
# Reorder the DataFrame based on the ordered columns
merged_df4=merged_df4[['college', 'Student_Count','Total_Assignment_count_Assignment_Goal_Setting','Acceptance_Rate_Assignment_Goal_Setting',
                'Accepted_Assignment_Goal_Setting','Rejected_Assignment_Goal_Setting','Under_Review_Assignment_Goal_Setting','Total_Assignment_count_Assignment_SMART_goals',
                'Acceptance_Rate_Assignment_SMART_goals','Total_Assignment_count_Assignment_CV_Resume','Acceptance_Rate_Assignment_CV_Resume',
                'Accepted_Assignment_CV_Resume','Rejected_Assignment_CV_Resume','Under_Review_Assignment_CV_Resume']]

In [ ]:
merged_df4

In [ ]:
# Assuming 'result1' is the DataFrame containing the data
merged_df4.fillna(0, inplace=True)


In [ ]:
data1=merged_df3[['college','Accepted', 'Rejected',
       'Under review', 'Total Assignment','Not Submitted','Acceptance Rate(in Percentage)']]

In [ ]:
data1

In [ ]:
# Assuming 'result1' is the DataFrame containing the data
data1.fillna(0, inplace=True)


In [ ]:
merged_df5=pd.merge(merged_df4,data1,on='college',how='inner')

In [ ]:
merged_df5

In [ ]:
# Assuming df is your final DataFrame
output_directory = r'C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\08 Support Code\02 Edu Team Notation Code\Output'

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

output_path = os.path.join(output_directory, 'Assignments_Overview.csv')
merged_df5.to_csv(output_path, index=False)

# Part E

In [ ]:
import os
import pandas as pd

folder_path = r'C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 SheForSTEM\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\04 Incubator - Curriculum Cohort\Incubator 6.0 UK\data_files\04_Source_Kalpana Assignment Review'   # Give your folder path here
files = os.listdir(folder_path)
csv_files = [file for file in files if file.endswith('.csv')]

dataframes_list = []  # List to store the extracted DataFrames
assignment_files_map = {}  # Dictionary to store the mapping of assignment file names with DataFrame variables
dataframe_names = []  # List to store the names of the assigned DataFrames

for i, file in enumerate(csv_files, 1):
    file_path = os.path.join(folder_path, file) 
    df = pd.read_csv(file_path)
    dataframe_name = f'data{i}'  # Variable name assigned to the DataFrame
    globals()[dataframe_name] = df  # Assign each DataFrame to variables data1, data2, data3, and so on
    dataframes_list.append(df)  
    dataframe_names.append(dataframe_name)  # Append the assigned DataFrame name to the list
    assignment_files_map[dataframe_name] = file  # Map assignment file names with DataFrame variables

# Print out the assignment file names associated with the DataFrames
print("Assignment file names associated with the DataFrames:")
for dataframe_var, file_name in assignment_files_map.items():
    print(f"{dataframe_var} is associated with file: {file_name}")


In [ ]:
assignment_files_map

In [ ]:
a=assignment_files_map.values()
b=list(a)
print(b)

In [ ]:
dataframe_names

In [ ]:
map_count = len(assignment_files_map)

In [ ]:
# Create a list comprehension to dynamically generate the list final_data
final_data = [globals()[f'data{i}'] for i in range(1, map_count + 1)]

In [ ]:
###Put the names that are present in the dataframe_names manually here in accordance to the name present number of file name ###
################################################################################################################################

num_date_columns_list = []  # List to store the count of date columns for each file

for data_file in final_data:
    data = {}
    date_column_count = 0

    for key, value in data_file.items():
        if key.startswith('data/') and key.endswith('/date/$date'):
            data[key] = value
            date_column_count += 1

    num_date_columns_list.append(date_column_count)

# Print the count of date columns for each data file
for i, count in enumerate(num_date_columns_list, start=1):
    print(f"Count of columns containing the date format in data file {i}: {count}")




In [ ]:
# Define the number of empty dictionaries you want in the list
num_dicts = map_count 

# Creating a list comprehension with empty dictionaries
data_list = [{} for _ in range(num_dicts)]


In [ ]:
dataframes = final_data
#data_list = [{}, {}, {}, {},{},{},{},{}]   #Add the number of {} in accordance with the number of data files that are 
                                              #imported that are mentioned in the data_frames names.
# Replace with the actual num_date_columns_list
num_date_columns_list = num_date_columns_list  

# Iterate through each DataFrame and extract the data into separate dictionaries using a loop
for i, (df, data, num_date_columns) in enumerate(zip(dataframes, data_list, num_date_columns_list), start=1):
    # Clear the dictionary before adding new data
    data.clear()  
    num_date_columns = num_date_columns 
    # Clear the dictionary before adding new data
    data = {}  
    for j in range(num_date_columns):
        key = f'mixed_date{j+1}'
        data[key] = df[f'data/{j}/date/$date']
        
        
    # Update the data list with the extracted data dictionary for the respective DataFrame
    data_list[i-1] = data  

# Now, data_list[0], data_list[1], data_list[2], data_list[3], data_list[4], data_list[5], and data_list[6] will contain the extracted data


In [ ]:
len(dataframe_names)

In [ ]:
# Create an empty list to store the DataFrames
dataframes = []

# Iterate through the data_list to create DataFrames
for i in range(len(dataframe_names)):
    # Create DataFrame from each data dictionary
    df = pd.DataFrame(data_list[i])
    # Append the created DataFrame to the list
    dataframes.append(df)  

# Print the DataFrames to verify
for i, df in enumerate(dataframes, start=1):
    print(f"DataFrame {i}:")
    print(df.head())
    #print('\n')


In [ ]:
# Iterate through each DataFrame in the list and convert the mixed format date columns to datetime objects
for df in dataframes:
    for col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        
# Iterate through each DataFrame in the list and perform the specified operation
for i, df in enumerate(dataframes, start=1):
    df['Latest_Submission'] = df.max(axis=1, skipna=True)
    #print(f"Latest Submission for DataFrame {i}:\n{df['Latest_Submission'].head()}\n")


In [ ]:
def get_previous_date(row):
    max_date_index = row.values.argmax()
    previous_date = row.iloc[max_date_index - 2] if max_date_index > 0 else pd.NaT
    return previous_date

# Iterate through each DataFrame in the list and apply the get_previous_date function
for i, df in enumerate(dataframes, start=1):
    df['Previous_Submission_Date'] = df.apply(get_previous_date, axis=1)
    #print(f"Previous Submission Dates for DataFrame {i}:\n{df['Previous_Submission_Date'].head()}\n")


In [ ]:
# Create an empty list to store the average time differences
average_time_differences = []

# Iterate through each DataFrame in the list
for i, df in enumerate(dataframes, start=1):
    # Calculate the difference between the Latest Submission and Previous Submission Date 
    df['Submission_Date_Difference'] = ((df['Latest_Submission'] - df['Previous_Submission_Date']).dt.days * 24)
    
    # Calculate the average time difference
    average_time_second_last = df['Submission_Date_Difference'].mean().round()
    
    # Append the average time difference to the list
    average_time_differences.append(average_time_second_last)
    
    # Print the head of the DataFrame including the calculated difference
    print(f"Difference in Submission Dates for DataFrame {i}:\n{df[['Previous_Submission_Date', 'Latest_Submission', 'Submission_Date_Difference']].head()}\n")

# Calculate the average of all the average time differences
overall_average_time_difference = (sum(average_time_differences) / len(average_time_differences)).round()
#print(f"Overall Average Time Difference: {overall_average_time_difference} hours\n")


# 🕐 Average Time 🕐

In [ ]:
print(f"Overall Average Time Difference: {overall_average_time_difference} hours\n")

# Part E - 1

In [ ]:


# Your existing code to create DataFrames remains the same
dataframes = []
for i in range(len(dataframe_names)):  
    df = pd.DataFrame(data_list[i])  
    dataframes.append(df)  

# Convert mixed format date columns to datetime objects
for df in dataframes:
    for col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Obtain the Latest Submission column
for i, df in enumerate(dataframes, start=1):
    df['Latest_Submission'] = df.max(axis=1, skipna=True)
    df['Latest_Submission'] = df['Latest_Submission'].astype(str)

# Split the Latest Submission column
for i, df in enumerate(dataframes, start=1):
    df['Latest_Submission_split'] = df['Latest_Submission'].str.split(' ', n=1).str[0]

# Prepare to save updated DataFrames to new CSV files
for i, df in enumerate(final_data, start=1):
    df['Latest_Submission'] = dataframes[i-1]['Latest_Submission']
    df['Latest_Submission_split'] = dataframes[i-1]['Latest_Submission_split']
    new_file_path = f'updated_file_{i}.csv'
    df.to_csv(new_file_path, index=False)

# Define the number of CSV files based on the length of the 'dataframes' list
number_of_files = len(dataframe_names)

# Determine the Previous Dates to the Latest Dates in the 'Latest_Submission_split' Column
latest_dates = []

for i in range(1, number_of_files + 1):
    file_path = f'updated_file_{i}.csv'  
    df = pd.read_csv(file_path)

    if 'Latest_Submission_split' in df.columns:
        latest_date = pd.to_datetime(df['Latest_Submission_split']).max().strftime('%Y-%m-%d')
        previous_date = (pd.to_datetime(latest_date) - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
        previous_date_to_date = (pd.to_datetime(latest_date) - pd.DateOffset(days=2)).strftime('%Y-%m-%d')

        # Add all relevant dates
        latest_dates.append(latest_date)
        latest_dates.append(previous_date)
        latest_dates.append(previous_date_to_date)
    else:
        print(f"'Latest_Submission_split' column does not exist in DataFrame {i}")

# Ensure that latest_dates is unique and sorted
sorted_target_dates = sorted(list(set(latest_dates)))

# You can manually define target_dates here or use sorted_target_dates
target_dates = sorted_target_dates  # Use all unique dates

# Process Status Counts based on Specific Target Dates for Multiple CSV Files
status_counts = []

for i in range(1, number_of_files + 1):  
    file_path = f'updated_file_{i}.csv'  
    df = pd.read_csv(file_path)

    status_counts_for_date = []  

    for target_date in target_dates:
        if 'Latest_Submission_split' in df.columns:
            filtered_data = df[pd.to_datetime(df['Latest_Submission_split']).dt.date == pd.to_datetime(target_date).date()]
            status_count = filtered_data['status'].value_counts().to_dict()
            status_counts_for_date.append(status_count)
            print(f"For preparing for Assignment - DataFrame {i} - Date {target_date}:")
            print(status_count)
        else:
            print(f"'Latest_Submission_split' column does not exist in DataFrame {i}")

    status_counts.append(status_counts_for_date)

# Create a list to store the tabular data
tabular_data = []

# Function to calculate the count of items under review
def get_under_review_count(status_count_dict):
    return status_count_dict.get('under review', 0)

# Iterate through each DataFrame and target date
for i in range(len(status_counts)):
    for j in range(len(target_dates)):
        if status_counts[i][j]:  # Checking if status count exists for the target date
            reviewed_count = status_counts[i][j].get('reviewed', 0)
            under_review_count = get_under_review_count(status_counts[i][j])
            rejected_count = status_counts[i][j].get('rejected', 0)
            tabular_data.append({
                'DataFrame': i + 1,
                'Assignment_file_names': b[i],
                'Date': target_dates[j],
                'Accepted': reviewed_count,
                '24h_New_Submits': under_review_count,
                'Rejected': rejected_count,
                'Total_checked': (reviewed_count + rejected_count)
            })

# Create a DataFrame from the tabular data
tabular_df = pd.DataFrame(tabular_data)




In [ ]:
tabular_df.head(10)

# Count Code of the daily updated Assignment Files:

In [ ]:
#Used the same code as above to extract the mixed date formats columns of the format 'data/{i}/date/$date'
# Create an empty list to store the DataFrames
dataframes = []

# Iterate through the data_list to create DataFrames
for i in range(len(dataframe_names)):  
    df = pd.DataFrame(data_list[i])  
    dataframes.append(df)  

# Print the DataFrames to verify
for i, df in enumerate(dataframes, start=1):
    print(f"DataFrame {i}:")
    print(df.head())
    print('\n')


In [ ]:
#Code to obtain the columns with latest submissions date i.e present in the latest updated column of the format 'data/{i}/date/$date' from the all the mixed data format column 
# Iterate through each DataFrame in the list and convert the mixed format date columns to datetime objects
for df in dataframes:
    for col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        
# Iterate through each DataFrame in the list and perform the specified operation
for i, df in enumerate(dataframes, start=1):
    df['Latest_Submission'] = df.max(axis=1, skipna=True)
    
    # Convert the "Latest Submission" column to string datatype
    df['Latest_Submission'] = df['Latest_Submission'].astype(str)
    
    print(f"Latest Submission for DataFrame {i}:\n{df['Latest_Submission'].head()}\n")


In [ ]:
# Specify the DataFrame index for DataFrame 1 (assuming it is the first DataFrame in the list)
df_index = 0

# Split the "Latest Submission for DataFrame 1" column rows containing datetime data after the first space
dataframes[df_index]['Latest_Submission_split'] = dataframes[df_index]['Latest_Submission'].str.split(' ', n=1).str[0]

# Print the modified DataFrame 1 with the split datetime data
print("Latest Submission for DataFrame 1 with Datetime Data after First Space:")
print(dataframes[df_index][['Latest_Submission', 'Latest_Submission_split']].head())


In [ ]:
# Iterate through each DataFrame in the list
for i, df in enumerate(dataframes, start=1):
    # Split the "Latest Submission" column rows containing datetime data after the first space
    df['Latest_Submission_split'] = df['Latest_Submission'].str.split(' ', n=1).str[0]
    
    # Print the modified DataFrame with the split datetime data
    print(f"Latest Submission for DataFrame {i} with Datetime Data after First Space:")
    print(df[['Latest_Submission', 'Latest_Submission_split']].head())
    print()  # Add a new line for better readability


In [ ]:
# Iterate through each DataFrame in the list
for i, df in enumerate(final_data, start=1):
    # Update the "Latest Submission" column in each DataFrame with the split data
    df['Latest_Submission'] = dataframes[i-1]['Latest_Submission']
    df['Latest_Submission_split'] = dataframes[i-1]['Latest_Submission_split']

    # Save the updated DataFrame to a new CSV file
    new_file_path = f'updated_file_{i}.csv'  # Replace with the desired file naming convention
    df.to_csv(new_file_path, index=False)

In [ ]:
df['status'].unique()

In [ ]:
# Define the number of CSV files based on the length of the 'dataframes' list
number_of_files = len(dataframe_names)

# Determine the Previous Dates to the Latest Dates in the 'Latest_Submission_split' Column
latest_dates = []

for i in range(1, number_of_files + 1):
    file_path = f'updated_file_{i}.csv'  
    df = pd.read_csv(file_path)
    
    if 'Latest_Submission_split' in df.columns:
        latest_date = pd.to_datetime(df['Latest_Submission_split']).max().strftime('%Y-%m-%d')
        previous_date = (pd.to_datetime(df['Latest_Submission_split']).max() - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
        previous_date_to_date = (pd.to_datetime(df['Latest_Submission_split']).max() - pd.DateOffset(days=2)).strftime('%Y-%m-%d')
        
        latest_dates.append(previous_date)
        latest_dates.append(previous_date_to_date)
        
    else:
        print(f"'Latest_Submission_split' column does not exist in DataFrame {i}")

sorted_target_dates = sorted(list(set(latest_dates)))
target_dates = sorted_target_dates[-1:]

################################################
#Use this to type dates yourself else it will take the date previous to the latest today's date
#target_dates=['2024-05-19','2024-05-20']
##################################################


# Process Status Counts based on Specific Target Dates for Multiple CSV Files
status_counts = []

for i in range(1, number_of_files + 1):  
    file_path = f'updated_file_{i}.csv'  
    df = pd.read_csv(file_path)

    status_counts_for_date = []  

    for target_date in target_dates:
        if 'Latest_Submission_split' in df.columns:
            filtered_data = df[pd.to_datetime(df['Latest_Submission_split']).dt.date == pd.to_datetime(target_date).date()]
            status_count = filtered_data['status'].value_counts().to_dict()
            status_counts_for_date.append(status_count)
            print(f"For preparing for Assignment - DataFrame {i} - Date {target_date}:")
            print(status_count)
        else:
            print(f"'Latest_Submission_split' column does not exist in DataFrame {i}")

    status_counts.append(status_counts_for_date)

# 'status_counts' now contains the status counts for each DataFrame for each target date


In [ ]:
# Create a list to store the tabular data
tabular_data = []

# Function to calculate the count of items under review
def get_under_review_count(status_count_dict):
    under_review_count = status_count_dict.get('under review', 0)  # Assuming 'under_review' is the key for under review counts
    return under_review_count

# Iterate through each DataFrame and target date
for i in range(len(status_counts)):
    for j in range(len(target_dates)):
        if status_counts[i][j]:  # Checking if status count exists for the target date
            reviewed_count = status_counts[i][j].get('reviewed', 0)
            under_review_count = get_under_review_count(status_counts[i][j])
            rejected_count = status_counts[i][j].get('rejected', 0)
            tabular_data.append({
                'DataFrame': i+1,
                'Assignment_file_names':b[i],
                'Date': target_dates[j],
                'Accepted': reviewed_count,
                '24h_New_Submits': under_review_count,
                'Rejected': rejected_count,
                'Total_checked':(reviewed_count+rejected_count)

                
            })

# Create a DataFrame from the tabular data
tabular_df = pd.DataFrame(tabular_data)

# Display the tabular DataFrame
tabular_df
